<a href="https://colab.research.google.com/github/pbeens/python/blob/master/Western_AQ_ABQ_Courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program retrieves all the AQ and ABQ courses from the UWO website and stores them in an HTML file.

GitHub URL: https://github.com/pbeens/python/blob/master/Western_AQ_ABQ_Courses.ipynb

Colab URL: https://colab.research.google.com/drive/1DfF86AtXKKhniCdes0IDHIAaz75m4hvW 

In [ ]:
# imports
from bs4 import BeautifulSoup
import urllib.request

In [ ]:
# global variable(s)
url = 'https://www.aspire.uwo.ca/aq-courses/index.html'
term = '2021 Fall'
subjects = ['Business Studies', 'Co-Operative Education', 'Dramatic Arts', 
            'Family Studies', 'French as a Second Language', 
            'Guidance and Career Education', 'Honour Specialist', 
            'Integration of Information and Computer Technology in Instruction',
            'Intermediate Division', 'Junior', 'Kindergarten', 'Mathematics',
            'Music', 'Primary', 'Reading', 'Religious Education in Catholic Schools',
            'Senior Division', 'Special Education', 'Teacher Librarian',
            'Teaching Children with Communication Needs (Autism Spectrum Disorders)',
            'Teaching English Language Learners', 'Teaching Students Who are Blind/Low Vision',
            'Visual Arts']

# grab each subject URL
subject_links = []

html_page = urllib.request.urlopen(url)
soup = BeautifulSoup(html_page, "html.parser")

for link in soup.findAll('a', href=True):
    if link.text.strip() in subjects:
        subject_links.append('https://www.aspire.uwo.ca/aq-courses/' + link['href'])

# grab each course URL
all_courses = []
for link in subject_links:
  print(f'Grabbing courses from {link}... ')
  html_page = urllib.request.urlopen(link)
  soup = BeautifulSoup(html_page, "html.parser")

  # only find the links with publicCourseSearchDetails in the link
  for link in soup.findAll('a'):
    s = str(link.get('href'))
    if s.find('publicCourseSearchDetails') > 1:
      all_courses.append(s)

print('Done.')

In [ ]:
# populate course_urls_dict 
course_urls_dict = {}

for link in all_courses:

    html_page = urllib.request.urlopen(link)
    soup = BeautifulSoup(html_page, "html.parser")

    subject = soup.select_one('span[class*=title]').text.strip()
    
    # miscellaneous fixes
    subject = subject.replace('Business Studies - ', '')
    subject = subject.replace('Honour Specialist-', 'Honour Specialist -')
    if subject.find('Science General') == -1:
        subject = subject.replace('Honour Specialist - Science', 'Honour Specialist -')
    
    print(f'{subject}... Done.')
  
    course_urls_dict[subject] = link

print('Done.')

# check the courses dictionary
# for (k,v) in course_urls_dict.items():
#     print(f'{k}: {v}')   

In [ ]:
# create html file with course listings
file = './courses.html'
with open(file, 'w') as f:
  s = f'<HTML>\n<HEAD>\n\t<TITLE>{term}</TITLE>\n</HEAD>\n<BODY>\n'
  f.write(s)
  for course, url in course_urls_dict.items():
    f.write(f'\t<a href="{url}">{course}</a><br>\n')
  s = '</BODY>\n<HTML>'
  f.write(s)
f.close()
print(f'{file} created.')